# Incidenti aerei

In questa progetto, analizziamo gli incidenti aerei avvenuti nel corso degli anni, per cercare di capire quali sono le migliori compagnie con cui volare e quali sarebbe meglio evitare. Per questo, analizziamo ad esempio il rapporto tra incidenti e grandezza di una compagnia oppure, mediante tabelle, confrontiamo l'andamento del numero di incidenti in periodi di tempo diversi.

**Dataset**

Per la nostra analisi, abbiamo scelto di utilizzare due dataset diversi:
* [Incidenti aerei dal 1908 al 2009](https://www.kaggle.com/saurograndi/airplane-crashes-since-1908)
* [Incidenti aerei dal 1985 al 2014](https://github.com/fivethirtyeight/data/tree/master/airline-safety)

Il primo dataset contiene tutti gli incidenti avvenuti il quel periodo di tempo corredati da numerose informazioni come la località, la data e la rotta.

Il secondo dataset contiene invece dati già raggruppati: è infatti presente il numero di incidenti raggruppati per due diversi intervalli di anni (1985-1999 e 2000-2014) e su un numero più limitato di compagnie aeree, che sono da considerarsi come le principali compagnie. Abbiamo integrato questo dataset aggiungendo tre attributi: nazionalità e continente della compagnie e il suo essere o meno una compagnia di bandiera.

Per la nostra analisi, utilizzeremo nella prima parte il primo dataset per dare delle informazioni più generali riguardo gli incidenti (per esempio i luoghi degli incidenti aerei) ed iniziare a valutare l'affidabilità delle compagnie e dei mezzi aerei. Valutazione che continuerà nella seconda parte in cui utilizzeremo il secondo dataset a disposizione per approfondire alcuni temi.


In [1]:
#Importo librerie per l'analisi

import pandas as pd
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
import random
import warnings 
warnings.filterwarnings('ignore') #disable warnings: it doesn't print them in notebook
py.init_notebook_mode(connected=True)

## Parte Uno - Informazioni Generali e Introduzione alla Valutazione delle Compagnie

### Incidents and Fatalities per year

Questi due primi grafici offrono una panoramica generale sugli incidenti aerei occorsi dal 1908 al 2009.

Nello specifico, il primo grafico mostra il numero di incidenti per ogni anno mentre il secondo il numero di vittime.

In [2]:
#Numero di incidenti per anno dal 1908 al 2009 e valutazione, in base al loro numero, della "bontà" dell'anno

df3 = pd.read_csv('Airplane_Crashes_and_Fatalities_Since_1908.csv') #import dataset


#stimo degli intervalli per modificare il colore
bins = [0, 500, 1000, 1500, 2500]

#trasformo la data contenuta come stringa nel dataset in formato data 
df3['Date'] = pd.to_datetime(df3['Date'])
#a questo punto ricavo solo l'anno
df3['Year'] = df3['Date'].map(lambda x:x.year)


#Function to draw charts
def drawNumberIncidentsOrFatalities(operation):
    #raggruppo il dataset per anno, in modo da poter verificare la somma di tutti gli incidenti in un anno
    title = ''
    colo = ''
    if(operation == 'incidents'):
        crash_per_year = df3.groupby([df3['Year']])['Fatalities'].count()
        title = 'Incidents per year'
        color = 'blue'
    elif(operation == 'fatalities'):
        crash_per_year = df3.groupby([df3['Year']])['Fatalities'].sum()
        title = 'Fatalities per year'
        color = 'red'

    #ricavo un array di anni, dalla tabella di anni-incidenti che ho ottenuto sopra
    crashes = crash_per_year.values
    years = crash_per_year.index

    
    trace = go.Bar(x = years, 
              y = crashes, marker=dict(
              color = color))    

    layout = go.Layout(
        title=title,
        xaxis=dict(title="Years"),
        yaxis =dict(title="Incidents", automargin=True))
    
    data=[trace]

    fig = go.Figure(data, layout)
    py.iplot(fig)


drawNumberIncidentsOrFatalities('incidents')
drawNumberIncidentsOrFatalities('fatalities')

### Airplane Crashes Since 1908

Il grafico riporta il numero di incidenti avvenuti in ogni nazione.

In [3]:
#Numero di incidenti, dal 1908 ad oggi, divisi per nazione

locations=["AFG", "ALB", "DZA", "ASM", "AND", "AGO", "AIA", "ATG", "ARG", "ARM", "ABW", "AUS", "AUT", "AZE", "BHM", "BHR", "BGD", "BRB", "BLR", "BEL", "BLZ", "BEN", "BMU", "BTN", "BOL", "BIH", "BWA", "BRA", "VGB", "BRN", "BGR", "BFA", "MMR", "BDI", "CPV", "KHM", "CMR", "CAN", "CYM", "CAF", "TCD", "CHL", "CHN", "COL", "COM", "COD", "COG", "COK", "CRI", "CIV", "HRV", "CUB", "CUW", "CYP", "CZE", "DNK", "DJI", "DMA", "DOM", "ECU", "EGY", "GBR", "SLV", "GNQ", "ERI", "EST", "ETH", "FLK", "FRO", "FJI", "FIN", "FRA", "PYF", "GAB", "GMB", "GEO", "DEU", "GHA", "GIB", "GRC", "GRL", "GRD", "GUM", "GTM", "GGY", "GNB", "GIN", "GUY", "HTI", "HND", "HKG", "HUN", "ISL", "IND", "IDN", "IRN", "IRQ", "IRL", "IMN", "ISR", "ITA", "JAM", "JPN", "JEY", "JOR", "KAZ", "KEN", "KIR", "KOR", "PRK", "KSV", "KWT", "KGZ", "LAO", "LVA", "LBN", "LSO", "LBR", "LBY", "LIE", "LTU", "LUX", "MAC", "MKD", "MDG", "MWI", "MYS", "MDV", "MLI", "MLT", "MHL", "MRT", "MUS", "MEX", "FSM", "MDA", "MCO", "MNG", "MNE", "MAR", "MOZ", "NAM", "NPL", "NLD", "NCL", "NZL", "NIC", "NGA", "NER", "NIU", "MNP", "NOR", "OMN", "PAK", "PLW", "PAN", "PNG", "PRY", "PER", "PHL", "POL", "PRT", "PRI", "QAT", "ROU", "RUS", "RWA", "KNA", "LCA", "MAF", "SPM", "VCT", "WSM", "SMR", "STP", "SAU", "SEN", "SRB", "SYC", "SLE", "SGP", "SXM", "SVK", "SVN", "SLB", "SOM", "ZAF", "SSD", "ESP", "LKA", "SDN", "SUR", "SWZ", "SWE", "CHE", "SYR", "TWN", "TJK", "TZA", "THA", "TLS", "TGO", "TON", "TTO", "TUN", "TUR", "TKM", "TUV", "UGA", "UKR", "ARE", "USA", "URY", "UZB", "VUT", "VEN", "VNM", "VGB", "WBG", "YEM", "ZMB", "ZWE"]
text= ["Afghanistan", "Albania", "Algeria", "American Samoa", "Andorra", "Angola", "Anguilla", "Antigua and Barbuda", "Argentina", "Armenia", "Aruba", "Australia", "Austria", "Azerbaijan", "Bahamas, The", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bermuda", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "British Virgin Islands", "Brunei", "Bulgaria", "Burkina Faso", "Burma", "Burundi", "Cabo Verde", "Cambodia", "Cameroon", "Canada", "Cayman Islands", "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Democratic Republic of the Congo", "Republic of the Congo", "Cook Islands", "Costa Rica", "Cote d'Ivoire", "Croatia", "Cuba", "Curacao", "Cyprus", "Czech Republic", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "Ecuador", "Egypt", "England", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Ethiopia", "Falkland Islands (Islas Malvinas)", "Faroe Islands", "Fiji", "Finland", "France", "French Polynesia", "Gabon", "Gambia, The", "Georgia", "Germany", "Ghana", "Gibraltar", "Greece", "Greenland", "Grenada", "Guam", "Guatemala", "Guernsey", "Guinea-Bissau", "Guinea", "Guyana", "Haiti", "Honduras", "Hong Kong", "Hungary", "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Isle of Man", "Israel", "Italy", "Jamaica", "Japan", "Jersey", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "Korea, North", "Korea, South", "Kosovo", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Macau", "Macedonia", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Micronesia, Federated States of", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Namibia", "Nepal", "Netherlands", "New Caledonia", "New Zealand", "Nicaragua", "Nigeria", "Niger", "Niue", "Northern Mariana Islands", "Norway", "Oman", "Pakistan", "Palau", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Puerto Rico", "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia", "Saint Martin", "Saint Pierre and Miquelon", "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Sint Maarten", "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Swaziland", "Sweden", "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Timor-Leste", "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Venezuela", "Vietnam", "Virgin Islands", "West Bank", "Yemen", "Zambia", "Zimbabwe"] 
usa=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachussets", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
colorscale=['Blackbody', 'Bluered', 'Blues', 'Earth', 'Electric', 'Greens', 'Grays', 'Hot', 'Jet', 'Picnic', 'Portland', 'Rainbow', 'RdBu', 'Reds', 'Viridis', 'YlGnBu', 'YlOrRd']


location = df3.dropna(subset=['Location']) 
location0 = location['Location'].values

states=[]
for i in location0:
    parts=i.split(",")
    if(len(parts)>=2):
        states.append(parts[len(parts)-1].lstrip())
            
my_count = pd.Series(states).value_counts()

z=[]
temp=0
for i in text:
    if(i=="United States"):
        for j in usa:
            if(sum(my_count.index.str.contains(j, regex=False))>=1):
                temp=temp+(my_count[j])
        z.append(temp)
    else:
        if(sum(my_count.index.str.contains(i, regex=False))>=1):
            z.append(my_count[i])
        else:
            z.append(0)
            
data = [ dict(
        type = 'choropleth',
        #colorscale = 'Rainbow',
        colorscale = colorscale[14],
        locations = locations,
        z = z,
        text = text,
 
        colorbar = dict(
            title = 'Incidents', 
            titlefont=dict(size=25),
            tickfont=dict(size=18))
      ) ]
 
# Define layout
layout = dict(
    title = 'Airplane Crashes Since 1908',
    titlefont = dict(size=40),
    geo = dict(
        showframe = True,
        showcoastlines = True,
        #projection = dict(type = 'equirectangular')
        projection = dict(type = 'orthographic')
              )
             )
 
# Plot
fig = dict( data=data, layout=layout )
#plot_url = py.offline.plot(fig, validate=False, filename='world.html') #apri in un'altra scheda
py.iplot(fig)

# COPIA BRE

In [21]:
#Numero di incidenti, dal 1908 ad oggi, divisi per nazione

locations=["AFG", "ALB", "DZA", "ASM", "AND", "AGO", "AIA", "ATG", "ARG", "ARM", "ABW", "AUS", "AUT", "AZE", "BHM", "BHR", "BGD", "BRB", "BLR", "BEL", "BLZ", "BEN", "BMU", "BTN", "BOL", "BIH", "BWA", "BRA", "VGB", "BRN", "BGR", "BFA", "MMR", "BDI", "CPV", "KHM", "CMR", "CAN", "CYM", "CAF", "TCD", "CHL", "CHN", "COL", "COM", "COD", "COG", "COK", "CRI", "CIV", "HRV", "CUB", "CUW", "CYP", "CZE", "DNK", "DJI", "DMA", "DOM", "ECU", "EGY", "GBR", "SLV", "GNQ", "ERI", "EST", "ETH", "FLK", "FRO", "FJI", "FIN", "FRA", "PYF", "GAB", "GMB", "GEO", "DEU", "GHA", "GIB", "GRC", "GRL", "GRD", "GUM", "GTM", "GGY", "GNB", "GIN", "GUY", "HTI", "HND", "HKG", "HUN", "ISL", "IND", "IDN", "IRN", "IRQ", "IRL", "IMN", "ISR", "ITA", "JAM", "JPN", "JEY", "JOR", "KAZ", "KEN", "KIR", "KOR", "PRK", "KSV", "KWT", "KGZ", "LAO", "LVA", "LBN", "LSO", "LBR", "LBY", "LIE", "LTU", "LUX", "MAC", "MKD", "MDG", "MWI", "MYS", "MDV", "MLI", "MLT", "MHL", "MRT", "MUS", "MEX", "FSM", "MDA", "MCO", "MNG", "MNE", "MAR", "MOZ", "NAM", "NPL", "NLD", "NCL", "NZL", "NIC", "NGA", "NER", "NIU", "MNP", "NOR", "OMN", "PAK", "PLW", "PAN", "PNG", "PRY", "PER", "PHL", "POL", "PRT", "PRI", "QAT", "ROU", "RUS", "RWA", "KNA", "LCA", "MAF", "SPM", "VCT", "WSM", "SMR", "STP", "SAU", "SEN", "SRB", "SYC", "SLE", "SGP", "SXM", "SVK", "SVN", "SLB", "SOM", "ZAF", "SSD", "ESP", "LKA", "SDN", "SUR", "SWZ", "SWE", "CHE", "SYR", "TWN", "TJK", "TZA", "THA", "TLS", "TGO", "TON", "TTO", "TUN", "TUR", "TKM", "TUV", "UGA", "UKR", "ARE", "USA", "URY", "UZB", "VUT", "VEN", "VNM", "VGB", "WBG", "YEM", "ZMB", "ZWE"]
text= ["Afghanistan", "Albania", "Algeria", "American Samoa", "Andorra", "Angola", "Anguilla", "Antigua and Barbuda", "Argentina", "Armenia", "Aruba", "Australia", "Austria", "Azerbaijan", "Bahamas, The", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bermuda", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "British Virgin Islands", "Brunei", "Bulgaria", "Burkina Faso", "Burma", "Burundi", "Cabo Verde", "Cambodia", "Cameroon", "Canada", "Cayman Islands", "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Democratic Republic of the Congo", "Republic of the Congo", "Cook Islands", "Costa Rica", "Cote d'Ivoire", "Croatia", "Cuba", "Curacao", "Cyprus", "Czech Republic", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "Ecuador", "Egypt", "England", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Ethiopia", "Falkland Islands (Islas Malvinas)", "Faroe Islands", "Fiji", "Finland", "France", "French Polynesia", "Gabon", "Gambia, The", "Georgia", "Germany", "Ghana", "Gibraltar", "Greece", "Greenland", "Grenada", "Guam", "Guatemala", "Guernsey", "Guinea-Bissau", "Guinea", "Guyana", "Haiti", "Honduras", "Hong Kong", "Hungary", "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Isle of Man", "Israel", "Italy", "Jamaica", "Japan", "Jersey", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "Korea, North", "Korea, South", "Kosovo", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Macau", "Macedonia", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Micronesia, Federated States of", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Namibia", "Nepal", "Netherlands", "New Caledonia", "New Zealand", "Nicaragua", "Nigeria", "Niger", "Niue", "Northern Mariana Islands", "Norway", "Oman", "Pakistan", "Palau", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Puerto Rico", "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia", "Saint Martin", "Saint Pierre and Miquelon", "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Sint Maarten", "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Swaziland", "Sweden", "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Timor-Leste", "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Venezuela", "Vietnam", "Virgin Islands", "West Bank", "Yemen", "Zambia", "Zimbabwe"] 
usa=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachussets", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
colorscale=['Blackbody', 'Bluered', 'Blues', 'Earth', 'Electric', 'Greens', 'Grays', 'Hot', 'Jet', 'Picnic', 'Portland', 'Rainbow', 'RdBu', 'Reds', 'Viridis', 'YlGnBu', 'YlOrRd']


location = df3.dropna(subset=['Location']) 
location0 = location['Location'].values

states=[]
for i in location0:
    parts=i.split(",")
    if(len(parts)>=2):
        states.append(parts[len(parts)-1].lstrip())
            
my_count = pd.Series(states).value_counts()

scl = [[0.0, 'rgb(230, 230, 255)'],[0.001, 'rgb(218, 218, 255)'],[0.05, 'rgb(150, 150, 255)'],[0.1, 'rgb(128, 128, 255)'],
            [0.15, 'rgb(77, 77, 255)'],[0.2, 'rgb(40, 40, 255)'],[1.0, 'rgb(30, 30, 130)']]

z=[]
temp=0
for i in text:
    if(i=="United States"):
        for j in usa:
            if(sum(my_count.index.str.contains(j, regex=False))>=1):
                temp=temp+(my_count[j])
        z.append(temp)
    else:
        if(sum(my_count.index.str.contains(i, regex=False))>=1):
            z.append(my_count[i])
        else:
            z.append(0)
            
data = [ dict(
        type = 'choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = locations,
        z = z,
        text = text,
 
        colorbar = dict(
            title = 'Incidents', 
            titlefont=dict(size=25),
            tickfont=dict(size=18))
      ) ]
 
# Define layout
layout = dict(
    title = 'Airplane Crashes Since 1908',
    titlefont = dict(size=40),
    geo = dict(
        showframe = True,
        showcoastlines = True,
        #projection = dict(type = 'equirectangular')
        projection = dict(type = 'orthographic')
              )
             )
 
# Plot
fig = dict( data=data, layout=layout )
#plot_url = py.offline.plot(fig, validate=False, filename='world.html') #apri in un'altra scheda
py.iplot(fig)

### USA Airplane Crashes Since 1908

In questo grafico sono riportati gli incidenti avvenuti negli Stati Uniti, che rappresentano lo stato con il maggior numero di incidenti aerei.

In [5]:

z=[]
for i in usa:
    if(sum(my_count.index.str.contains(i, regex=False))>=1):
        z.append(my_count[i])
    else:
        z.append(0)

scl = [[0.0, 'rgb(230, 230, 255)'],[0.2, 'rgb(179, 179, 255)'],[0.4, 'rgb(128, 128, 255)'],
            [0.6, 'rgb(77, 77, 255)'],[0.8, 'rgb(40, 40, 255)'],[1.0, 'rgb(0, 0, 255)']]

code=["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = code,
        z = z,
        locationmode = 'USA-states',
        text = usa,
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Incidents",
            titlefont=dict(size=25),
            tickfont=dict(size=18))
        ) ]

layout = dict(
        title = 'USA Airplane Crashes Since 1908',
        titlefont = dict(size=40),
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig)


### 25 Airplane Models With Most Incidents

Nei due grafici seguenti sono riportati i 25 modelli di aereo che hanno subito il maggior numero di incidenti.

Nel primo grafico abbiamo il numero degli incidenti. Nel secondo, troviamo il valore in percentuale che permette di capire quanto un determinato modello sia coinvolto in incidenti.

In [6]:
from collections import Counter 
type_list=df3['Type'].dropna()
airplane_type= list(reversed(Counter(type_list).most_common(25)))
type_index = [type_[0] for type_ in airplane_type]
type_val = [type_[1] for type_ in airplane_type]

trace = go.Bar(x=type_val, y=type_index, orientation = 'h')

data = [trace]

layout = go.Layout(
    title="25 airplane model with most incidents",
    xaxis=dict(title="Number of incidents"),
    yaxis =dict(automargin=True),
    barmode="stack",
    font=dict(size=11))

fig = go.Figure(data, layout)
        
py.iplot(fig)


# Percentuale del modello

perc_mod=np.array(type_val)/len(type_list)*100
trace = go.Bar(x=perc_mod, y=type_index, orientation = 'h')

data = [trace]

layout = go.Layout(
    title="% Incidents Of 25 Airplane Models With Most Incidents",
    xaxis=dict(title="% of incidents"),
    yaxis =dict(automargin=True),
    barmode="stack",
    font=dict(size=11))

fig = go.Figure(data, layout)
        
py.iplot(fig)

In [7]:
#Fatalities Aboard
np.seterr(divide='ignore', invalid='ignore')
Fatalities = df3.dropna(subset=['Fatalities']) 
Aboard_Fatalities = Fatalities.dropna(subset=['Aboard']) 
aboard = Aboard_Fatalities["Aboard"].values

fatalities = Aboard_Fatalities["Fatalities"].values
survivors = aboard-fatalities

colors=survivors/aboard

print(f"Numero di morti: {np.sum(fatalities)}, numero di sopravvissuti: {np.sum(survivors)}")
data = [ 
    go.Scatter(x=fatalities, y=survivors, mode="markers",
               marker=dict(
                   size=6, 
                   color = colors,
                   colorscale='Viridis',
                   showscale=True
               ),
               name="data"),
]

layout = go.Layout(title = 'Survivors vs Fatalities', xaxis = {'title': 'Fatalities'}, yaxis = {'title': 'Survivors'}, barmode='overlay')
fig=go.Figure(data=data, layout=layout)

py.iplot(fig)

Numero di morti: 105358.0, numero di sopravvissuti: 39193.0


In [8]:
time = (df3.dropna(subset=['Time']))["Time"].values
ftr = [3600,60,1]
for i in range(len(time)):
    try:
        temp=sum([a*b for a,b in zip(ftr, map(int,time[i].split(':')))])
        if (temp>86400):
            temp=random.randint(0, 84600)
        time[i]=temp
    except:
        time[i]=random.randint(0, 84600)
incident_time= np.array(Counter(time).most_common(len(time)))
incident_time=incident_time[incident_time[:,0].argsort()]

data = [ 
    go.Scatter(x=(incident_time[:,0]/3600), y=incident_time[:,1], mode="lines",  marker = {'color': 'rgb(0,0,255)', 'size': 5, 'symbol': 'circle', 'opacity': 1.0}, name="data"),
]

layout = go.Layout(title = 'Incidents vs Time', xaxis = {'title': 'Time in hour'}, yaxis = {'title': 'Incidents number'}, barmode='overlay')
fig=go.Figure(data=data, layout=layout)

py.iplot(fig)

## Parte Due - Valutazione dell'Affidabilità e della Sicurezza delle Principali Compagnie

In [9]:
#Importo dataset

df = pd.read_csv("airline-safety.csv")
dfAggiuntivo = pd.read_csv("dataset-aggiuntivo-csv.csv")

print (df)
print(dfAggiuntivo)

#importo i dati e li metto nei rispettivi array

airlines = df['airline'].values
ask = df['avail_seat_km_per_week'].values
incident85_99 = df['incidents_85_99'].values
incident00_14 = df['incidents_00_14'].values

                       airline  avail_seat_km_per_week  incidents_85_99  \
0                   Aer Lingus               320906734                2   
1                    Aeroflot*              1197672318               76   
2        Aerolineas Argentinas               385803648                6   
3                  Aeromexico*               596871813                3   
4                   Air Canada              1865253802                2   
5                   Air France              3004002661               14   
6                   Air India*               869253552                2   
7             Air New Zealand*               710174817                3   
8             Alaska Airlines*               965346773                5   
9                     Alitalia               698012498                7   
10          All Nippon Airways              1841234177                3   
11                   American*              5228357340               21   
12           Austrian Air

### Incidents VS ASK (1)

Iniziamo la seconda parte dell'analisi confrontando il numero di incidenti totali con la grandezza di una compagnia.

Per misurare la grandezza di una compagnia, utilizziamo una particolare unità di misura chiamata ***ASK*** che rappresenta il numero di posti che una compagnia mette a disposizione dei suoi passeggeri settimanalmente moltiplicata per il numero di chilometri percorsi, nello stesso intervallo di tempo, dalla compagnia.

Purtroppo, a causa delle mancanza di alcune informazioni, dobbiamo assumere che nell'intervallo preso in esame (1985-2014) le dimensioni delle compagnie siamo rimaste immutate: mancano infatti dati precisi per quanto riguarda la prima metà del dataset (quindi 1985-1999).

**Commento al grafico e considerazioni**

Dal grafico, possiamo notare una crescita sensata del numero di incidenti al crescere dell'ASK: il coefficiente di correlazione tra le due variabile prese in esame è pari a **0.472**, meno di quanto forse ci si potrebbe aspettare ma comunque buono consideranto il "difetto" di cui soffre il dataset esposto poco più sopra.

In [10]:
#Scatter plot che confronta gli incidenti di tutti gli anni con ask

data = []

incidentSum = incident85_99+incident00_14

trace = go.Scatter(x = ask, y = incidentSum,
                   mode = 'markers', marker = {'color': 'rgb(255,0,255)', 'size': 5, 'symbol': 'circle', 'opacity': 1.0},
                   text = airlines,
                   name = "Incidents 85-99"
                  ) 
data.append(trace)


layout = go.Layout(title = 'Incidents vs ASK', xaxis = {'title': 'ASK'}, yaxis = {'title': 'Number of incidents'}, barmode='overlay')
fig=go.Figure(data=data, layout=layout)

py.iplot(fig)

#print(np.corrcoef(incidentSum, ask)) #coefficiente correlazione


### Incidents VS ASK (2)

Eseguiamo un'analisi molto simile alla precedente ma, questa volta, non utilizziamo il numero totale di incidenti ma li teniamo separati (in base al periodo) così come ci vengono forniti nel dataset.

**Commento al grafico e considerazioni**

Dal grafico notiamo che la tendenza rimane simile al precedente, soprattutto per quanto riguarda gli incidenti del periodo 2000-2014 (sui quali l'informazione relativa all'ASK è più precisa).

L'impressione è poi confermata dal calcolo del coefficiente di correlazione: 
* per il periodo 1985-1999 (dati ASK non precisi) otteniamo un coefficiente di **0.28** che dimostra la poca correlazione tra il numero di incidenti e un ASK che è calcolato su flotte più recenti (che potrebbero nel frattempo essere cresciute così come rimpicciolite)
* per il periodo 2000-2014 (dati ASK precisi) otteniamo un coefficiente di **0.73** che dimostra una buona correlazione tra il numero di incidenti e la grandezza della compagnia: viene naturale pensare che più una compagnia opera voli più questa sia soggetta ad un numero di incidenti maggiori

In [11]:
#Scatter plot che confronta gli incidenti (divisi in due blocchi in base all'anno) con ask

data = []

#scatter
trace = go.Scatter(x = ask, y = incident85_99,
                   mode = 'markers', marker = {'color': 'rgb(255,0,0)', 'size': 5, 'symbol': 'circle', 'opacity': 1.0},
                   text = airlines,
                   name = "Incidents 85-99"
                  ) 
data.append(trace)

trace = go.Scatter(x = ask, y = incident00_14,
                   mode = 'markers', marker = {'color': 'rgb(0,0,255)', 'size': 5, 'symbol': 'circle', 'opacity': 1.0},
                   text = airlines,
                   name = "Incidents 00-14"
                  ) 

data.append(trace)


layout = go.Layout(title = 'Incidents vs ASK', xaxis = {'title': 'ASK'}, yaxis = {'title': 'Number of incidents'}, barmode='overlay')
fig=go.Figure(data=data, layout=layout)

py.iplot(fig)

#print(np.corrcoef(incident85_99, ask)) //coefficiente di correlazione
#print(np.corrcoef(incident00_14, ask)) //coefficiente di correlazione



### Incidents 85-99 per airline

Vediamo in questo grafico il numero di incidenti totale che ogni compagnia ha avuto nel periodo 1985-1999.

In [12]:
#istogramma con il nome della compagnia e il numero di incidenti

dfSort = df.sort_values(['incidents_85_99', 'airline'], ascending=[1, 1]) #importo il dataset ordinato


sortedAirlines = dfSort['airline'].values
sortedIncident_85_99 = dfSort['incidents_85_99'].values

trace = go.Bar(x=sortedIncident_85_99, y=sortedAirlines, orientation = 'h')


data = [trace]

layout = go.Layout(
    title="Incidents 85-99 per airline",
    xaxis=dict(title="Number of incidents"),
    yaxis =dict(automargin=True),
    font=dict(size=5))

fig = go.Figure(data, layout)
        
py.iplot(fig)
        

### Incidents vs Fatal Incidents 85-99 per airline

In questo grafico troviamo, per ogni compagnia, il numero totale di incidenti nel periodo 1985-1999 diviso tra incidenti con vittime e incidenti senza vittime.


Vediamo prima il grafico con il numero di incidenti in assoluto e poi il grafico che mostra qual è la percentuale di incidenti senza vittime e con vittime.

In [13]:
#istogramma che mostra il numero di incidenti "normali" e incidenti fatali nell'intervallo 85-99 per compagnia

airlines = dfSort['airline'].values
incident85_99 = dfSort['incidents_85_99'].values
fatalIncident85_99 = dfSort['fatal_accidents_85_99'].values

notFatalIncident85_99 = incident85_99 - fatalIncident85_99
percentFatal = fatalIncident85_99 / incident85_99
number_of_incidents = sum(incident85_99)


listaIncidenti = [fatalIncident85_99, notFatalIncident85_99]

trace1 = go.Bar(x=fatalIncident85_99, y=airlines, orientation = 'h', name = "Fatal incidents")
trace2 = go.Bar(x=notFatalIncident85_99, y=airlines, orientation='h', name = "Not fatal incidents")


data = [trace2, trace1]

layout = go.Layout(
    title="Incidents vs Fatal Incidents 85-99 per airline",
    xaxis=dict(title="Number of incidents"),
    yaxis =dict(automargin=True),
    barmode="stack",
    font=dict(size=5) )

fig = go.Figure(data, layout)
        
py.iplot(fig)


#ordino in modo decrescente il numero degli incidenti fatali
ix = np.argsort(-percentFatal)
#ora effettuo l'effettivo ordinamento usando un array di incidi ordinati
sorted_percentFatal = percentFatal[ix]
sorted_airlines = airlines[ix]



trace1 = go.Bar(x=sorted_percentFatal, y=sorted_airlines, orientation = 'h', name = "Fatal incidents")
trace2 = go.Bar(x=1-sorted_percentFatal, y=sorted_airlines, orientation='h', name = "Not fatal incidents")

data = [trace2, trace1]

layout = go.Layout(
    title="Incidents vs Fatal Incidents 85-99 per airline",
    xaxis=dict(title="Number of incidents"),
    yaxis =dict(automargin=True),
    barmode="stack",
    font=dict(size=5) )

fig = go.Figure(data, layout)
        
py.iplot(fig)

### Incidents vs Fatal Incidents 00-14 per airline

In questo grafico troviamo, per ogni compagnia, il numero totale di incidenti nel periodo 2000-2014 diviso tra incidenti con vittime e incidenti senza vittime.

Vediamo prima il grafico con il numero di incidenti in assoluto e poi il grafico che mostra qual è la percentuale di incidenti senza vittime e con vittime.

In [14]:
#istogramma che mostra il numero di incidenti "normali" e incidenti fatali nell'intervallo 85-99 per compagnia
dfSort = df.sort_values(['incidents_00_14', 'airline'], ascending=[1, 1]) #importo il dataset ordinato
airlines = dfSort['airline'].values
incident00_14 = dfSort['incidents_00_14'].values
fatalIncident00_14 = dfSort['fatal_accidents_00_14'].values

notFatalIncident00_14 = incident00_14 - fatalIncident00_14
percentFatal = fatalIncident00_14 / incident00_14
number_of_incidents = sum(incident00_14)


listaIncidenti = [fatalIncident00_14, notFatalIncident00_14]

trace1 = go.Bar(x=fatalIncident00_14, y=airlines, orientation = 'h', name = "Fatal incidents")
trace2 = go.Bar(x=notFatalIncident00_14, y=airlines, orientation='h', name = "Not fatal incidents")


data = [trace2, trace1]

layout = go.Layout(
    title="Incidents vs Fatal Incidents 00-14 per airline",
    xaxis=dict(title="Number of incidents"),
    yaxis =dict(automargin=True),
    barmode="stack",
    font=dict(size=5) )

fig = go.Figure(data, layout)
        
py.iplot(fig)

#ordino in modo decrescente il numero degli incidenti fatali
ix = np.argsort(-percentFatal)
#ora effettuo l'effettivo ordinamento usando un array di incidi ordinati
sorted_percentFatal = percentFatal[ix]
sorted_airlines = airlines[ix]



trace1 = go.Bar(x=sorted_percentFatal, y=sorted_airlines, orientation = 'h', name = "Fatal incidents")
trace2 = go.Bar(x=1-sorted_percentFatal, y=sorted_airlines, orientation='h', name = "Not fatal incidents")

data = [trace2, trace1]

layout = go.Layout(
    title="Incidents vs Fatal Incidents 85-99 per airline",
    xaxis=dict(title="Number of incidents"),
    yaxis =dict(automargin=True),
    barmode="stack",
    font=dict(size=5) )

fig = go.Figure(data, layout)
        
py.iplot(fig)

### Incidents per period

Nei due boxplot seguenti troviamo la distribuzione degli incidenti avuti da tutte le compagnie divisi in base ai periodi di tempo fornitici dal dataset.

**Commento al grafico e considerazioni**

Nel primo boxplot, riferito al periodo 1985-1999, abbiamo una mediana corrispondente a 4 incidenti. Tra gli outliers, spicca sicuramente Aeroflot che con 76 incidenti supera di oltre 50 casi l'outlier più vicino.

Nel secondo boxplot, riferito al periodo 2000-2014, la mediana si abbassa a 3 e si riduce anche il numero di outliers (passa da 5 a 3). Inoltre non si notano casi molto particolari di osservazioni fuori scala come avvenuto nel periodo precedente.

In [15]:
#boxplot per incidenti totali divisi per anni

incident85_99 = df['incidents_85_99'].values
incident00_14 = df['incidents_00_14'].values
airlines = df['airline'].values

trace1=go.Box(x=incident85_99, name="Incidents 1985-1999")
trace2=go.Box(x=incident00_14, name="Incidents 2000-2014")

data=[trace2, trace1]

layout=go.Layout(
    title="Incidents per period",
    xaxis=dict(title="Number of incidents"),
    yaxis =dict(automargin=True),
    annotations=[
        dict(
            x=76, y=1,
            xref='x', yref='y', text='Aeroflot',
            showarrow=True, arrowhead=6, ax=0, ay=-40, arrowcolor = "orange"
        ),
        dict(
            x=25, y=1,
            xref='x', yref='y', text='Ethiopian Airlines',
            showarrow=True, arrowhead=6, ax=90, ay=-50, arrowcolor = "orange"
        ),
        dict(
            x=24, y=1,
            xref='x', yref='y', text='Delta',
            showarrow=True, arrowhead=6, ax=0, ay=-60, arrowcolor = "orange"
        ),
        dict(
            x=21, y=1,
            xref='x', yref='y', text='American Airlines',
            showarrow=True, arrowhead=6, ax=0, ay=-80, arrowcolor = "orange"
        ),
        dict(
            x=19, y=1,
            xref='x', yref='y', text='United Airlines',
            showarrow=True, arrowhead=6, ax=0, ay=-120, arrowcolor = "orange"
        ),
        
        
        dict(
            x=24, y=0,
            xref='x', yref='y', text='Delta',
            showarrow=True, arrowhead=6, ax=50, ay=-40, arrowcolor = "blue"
        ),
        dict(
            x=17, y=0,
            xref='x', yref='y', text='American',
            showarrow=True, arrowhead=6, ax=50, ay=-40, arrowcolor = "blue"
        ),
        dict(
            x=14, y=0,
            xref='x', yref='y', text='United',
            showarrow=True, arrowhead=6, ax=0, ay=-40, arrowcolor = "blue"
        )
    ])

fig=go.Figure(data, layout)
py.iplot(fig)



### Fatal incidents per period

Nei due boxplot seguenti troviamo la distribuzione degli incidenti fatali (quindi con vittime) avuti da tutte le compagnie divisi in base ai periodi di tempo fornitici dal dataset.

**Commento al grafico e considerazioni**

Nel primo boxplot, riferito al periodo 1985-1999, abbiamo una mediana corrispondente a 1 incidente fatale. Tra gli outliers, Aeroflot mantiene il triste primato di compagnia con più incidenti (14), riuscendo però ad essere molto più vicino alle altre compagnie.

Nel secondo boxplot, riferito al periodo 2000-2014, la mediana è di 0 incidenti fatali, a dimostrazione dell'abbassamento degli incidenti aerei. E' presente inoltre un solo outlier (United) che è però superiore rispetto al baffo di una sola occorrenza.

In [16]:

fatalIncident00_14 = df['fatal_accidents_00_14'].values

trace1=go.Box(x=fatalIncident85_99, name="Incidents 1985-1999")
trace2=go.Box(x=fatalIncident00_14, name="Incidents 2000-2014")

data=[trace2, trace1]

layout=go.Layout(
    title="Fatal incidents per period",
    xaxis=dict(title="Number of incidents"),
    yaxis =dict(automargin=True),
    annotations=[
        dict(
            x=14, y=1,
            xref='x', yref='y', text='Aeroflot',
            showarrow=True, arrowhead=6, ax=0, ay=-40, arrowcolor = "orange"
        ),
        dict(
            x=12, y=1,
            xref='x', yref='y', text='Delta',
            showarrow=True, arrowhead=6, ax=0, ay=-60, arrowcolor = "orange"
        ),
        dict(
            x=8, y=1,
            xref='x', yref='y', text='United Airlines',
            showarrow=True, arrowhead=6, ax=0, ay=-120, arrowcolor = "orange"
        ),
        
        
        dict(
            x=3, y=0,
            xref='x', yref='y', text='United',
            showarrow=True, arrowhead=6, ax=0, ay=-40, arrowcolor = "blue"
        )
    ])

fig=go.Figure(data, layout)
py.iplot(fig)


### Fatalities per period

Nei due boxplot seguenti troviamo la distribuzione delle vittime di incidenti avuti da tutte le compagnie divisi in base ai periodi di tempo fornitici dal dataset.

**Commento al grafico e considerazioni**

Nel primo boxplot, riferito al periodo 1985-1999, abbiamo una mediana corrispondente a 48.5 vittime per compagnia. Tra gli outliers, China Airlines ha il maggior numero di vittime (535), seguita da Japan Airlines; notare che il baffo superiore arriva a 425 vittime, circa 100 in meno del primo outlier.

Nel secondo boxplot, riferito al periodo 2000-2014, la mediana è di 0 vittime per compagnia: anche in questo grafico è possibile notare le compagnie siano riuscite a ridurre il numero di vittime degli incidenti (riuscendo ad abbassare il numero di incidenti e dotando gli aeromobili di strumenti più sofisticati). Peggiorano però gli outliers: il più alto numero di vittime è di Malaysia Airlines (537) e aumentano anche in generale il numero di outliers (anche se la situazione è data dall'abbassamento della mediana e di conseguenza il baffo superiore si è abbassato.

In [17]:
#boxplot con le vittime di incidenti

fatalities85_99 = df['fatalities_85_99']
fatalities00_14 = df['fatalities_00_14'].values

trace1=go.Box(x=fatalities85_99, name="Fatalities 1985-1999")
trace2=go.Box(x=fatalities00_14, name="Fatalities 2000-2014")

data=[trace2, trace1]

layout=go.Layout(
    title="Fatalities per period",
    xaxis=dict(title="Number of incidents"),
    yaxis =dict(automargin=True),
    annotations=[
        dict(
            x=535, y=1,
            xref='x', yref='y', text='China Airlines',
            showarrow=True, arrowhead=6, ax=0, ay=-40, arrowcolor = "orange"
        ),
        dict(
            x=520, y=1,
            xref='x', yref='y', text='Japan Airlines',
            showarrow=True, arrowhead=6, ax=-50, ay=-60, arrowcolor = "orange"
        ),
        dict(
            x=225, y=0,
            xref='x', yref='y', text='China Airlines',
            showarrow=True, arrowhead=6, ax=0, ay=-40, arrowcolor = "blue"
        ),
        dict(
            x=283, y=0,
            xref='x', yref='y', text='Kenya Airways',
            showarrow=True, arrowhead=6, ax=0, ay=-70, arrowcolor = "blue"
        ),
        dict(
            x=337, y=0,
            xref='x', yref='y', text='Air France',
            showarrow=True, arrowhead=6, ax=0, ay=-40, arrowcolor = "blue"
        ),
        dict(
            x=416, y=0,
            xref='x', yref='y', text='American*',
            showarrow=True, arrowhead=6, ax=0, ay=-40, arrowcolor = "blue"
        ),
        dict(
            x=537, y=0,
            xref='x', yref='y', text='Malaysia Airlines',
            showarrow=True, arrowhead=6, ax=0, ay=-40, arrowcolor = "blue"
        )
    ])

fig=go.Figure(data, layout)
py.iplot(fig)


### Airline improvements

Confrontiamo il numero di incidenti totali occorsi alle singole compagnie per vedere se negli anni recenti, con il miglioramento delle norme di sicurezza e delle tecnologie, il numero di incidenti è calato, aumentato oppure rimasto invariato (se il numero è rimasto invariato, quindi la differenza è 0, la compagnia viene indicata come positiva).


**Commento al grafico e considerazioni**

Dalla tabella, vediamo che ci sono 40 compagnie che hanno migliorato (o per lo meno non peggiorato) la propria situazione, riducendo il numero di incidenti tra i due diversi periodi.

I miglioramenti più grandi sono avvenuti per due compagnie in particolare:
* Aeroflot: rappresenta la riduzione piu significativa, passando da 76 a soli 6 incidenti
* Ethiopian Airlines: riduce il numero di incidenti da 25 a solamente 5

La tendenza di riduzione degli incidenti era già stata evidenziata dai precedenti box-plot: sia il box che i baffi, nel periodo 2000-2014, si sono ridotti ed erano scomparsi gli outliers più alti: erano rimaste infatti solo compagnie americane che risultano essere quelle con ASK maggiore di tutto il dataset (quindi più probabilità di essere soggette a incidenti).


In [18]:
#tabella che mostra differenza tra incidenti prima e dopo

#airline
incidentDifferent = incident00_14 - incident85_99

trace = go.Table(
    header=dict(values=['Airline', '85-99 Incidents', '00-14 Incidents', 'Difference'],
                line = dict(color='#7D7F80'),
                fill = dict(color='#a1c3d1'),
                align = ['center']),
    cells=dict(values=[airlines,
                       incident85_99,
                       incident00_14,
                      incidentDifferent],
               line = dict(color='#7D7F80'),
               fill = dict(color=[['rgb(245,245,245)'], ['rgb(245,245,245)'], ['rgb(245,245,245)'],#unique color for column
                                #['rgba(0,250,0, 0.8)' if inc2<inc1 else 'rgba(250,0,0, 0.8)' for inc1, inc2 in zip(incident85_99, incident00_14)] ]),
                                 ['rgba(0,250,0, 0.8)' if inc <= 0 else 'rgba(250,0,0, 0.8)' for inc in incidentDifferent] ]),
                align = ['center']))

layout = go.Layout(
    title="Airlines improvements",
    autosize = True)

data = [trace]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = 'styled_table')


def numeroCompagnieMigliorate():
    migliorati = 0

    for inc1, inc2 in zip(incident85_99, incident00_14):
        if inc2 <= inc1:
            migliorati+=1 
    return migliorati

### Fatal Incidents Airlines Improvements

Confrontiamo il numero di incidenti fatali occorsi alle singole compagnie per vedere se negli anni recenti, con il miglioramento delle norme di sicurezza e delle tecnologie, il numero di incidenti è calato, aumentato oppure rimasto invariato (se il numero è rimasto invariato, quindi la differenza è 0, la compagnia viene indicata come positiva).

**Commento al grafico e considerazioni**

Dalla tabella, vediamo che ci sono 50 compagnie che hanno migliorato (o per lo meno non peggiorato) la propria situazione, riducendo il numero di incidenti fatali tra i due diversi periodi.

I miglioramenti più significativi sono avvenuti per:

Aeroflot: rappresenta la riduzione piu significativa (come già successo per gli incidenti totali), passando da 14 a un solo incidente

Delta: riduce il numero di incidenti da 12 a 2, con una differenza positiva di 10 incidenti

La tendenza di riduzione degli incidenti era già stata evidenziata dai precedenti box-plot: sia il box che i baffi, nel periodo 2000-2014, si sono ridotti ed erano scomparsi gli outliers più alti: erano rimaste infatti solo compagnie americane che risultano essere quelle con ASK maggiore di tutto il dataset (quindi più probabilità di essere soggette a incidenti).

In [19]:
df = pd.read_csv("airline-safety.csv")
airlines = df['airline'].values
fatalIncident85_99 = df['fatal_accidents_85_99'].values
fatalIncident00_14 = df['fatal_accidents_00_14'].values
fatalIncidentDifferent = fatalIncident00_14 - fatalIncident85_99

trace = go.Table(
    header=dict(values=['Airline', '85-99 Fatal Incidents', '00-14 Fatal Incidents', 'Difference'],
                line = dict(color='#7D7F80'),
                fill = dict(color='#a1c3d1'),
                align = ['center']),
    cells=dict(values=[airlines,
                       fatalIncident85_99,
                       fatalIncident00_14,
                      fatalIncidentDifferent],
               line = dict(color='#7D7F80'),
               fill = dict(color=[['rgb(245,245,245)'], ['rgb(245,245,245)'], ['rgb(245,245,245)'],#unique color for column
                                 ['rgba(0,250,0, 0.8)' if inc <= 0 else 'rgba(250,0,0, 0.8)' for inc in fatalIncidentDifferent] ]),
                align = ['center']))

layout = go.Layout(
    title="Fatalities Airlines improvements",
    autosize = True)

data = [trace]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = 'styled_table')



def numeroCompagnieMigliorate2():
    migliorati = 0

    for inc1, inc2 in zip(fatalIncident85_99, fatalIncident00_14):
        if inc2 <= inc1:
            migliorati+=1 
    print(migliorati)
    return migliorati